In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = '../data/cifar-10-batches-py'
print("1. Structure of CIFAR_DIR {0}".format(os.listdir(CIFAR_DIR)))

1. Structure of CIFAR_DIR ['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def loadData(filename):
    '''
    Read data from filename
    '''
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    '''
    CifarData is a wrapper that process CIFAR-10 Data
    '''
    def __init__(self, filenames, need_shuffle):
        '''
        The constructor takes in a list of files and read all data and labels into self._data member
        @param filenames - list of files that contains all data
        @param need_shuffle - a boolean flag indicating whether order of data needs to be shuffled. 
                              `True` of training set and `False` for testing set
        '''
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = loadData(filename)
            all_data.append(data)
            all_labels.append(labels)
        
        self._data = np.vstack(all_data) / 127.5 - 1
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
            # When we process the data by Mini Batch, this is a cursor indicating where
            # we are in the dataset
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        '''
        Get a random permutation of all indices between 0 and _num_examples, and apply 
        this random permutation of indices to `self._data` and `self._labels`.
        '''
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        '''
        Generate a batch of data for processing. For training set, if end index is larger 
        than `self._num_examples`, reshuffle the data and start over from index 0. For 
        testing set, if end index is out of bound, raise an exception.
        @param batch_size - Number of samples for the batch of data and labels to return
        '''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("Do not have enough testing samples for next batch")
                
        if end_indicator > self._num_examples:
            raise Exception("Batch size is larger than _num_examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    def reset(self):
        '''
        `_shuffle_data` and reset `_indicator` to 0
        '''
        self._shuffle_data()
        self._indicator = 0
    
    def getDataShape(self):
        return self._data.shape
    
    def getLabelShape(self):
        return self._labels.shape
    
print("2. Basic test for `loadData` and `CifarData` class.")
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
print("Train data shape = {0}, labels shape = {1}".
      format(train_data.getDataShape(), train_data.getLabelShape()))

test_data = CifarData(test_filenames, False)
print("Test data shape = {0}, labels shape = {1}".
      format(test_data.getDataShape(), test_data.getLabelShape()))

batch_data, batch_labels = train_data.next_batch(10)
print("Batch Data = {0}, Batch Labels = {1}".format(batch_data, batch_labels))

2. Basic test for `loadData` and `CifarData` class.
Train data shape = (50000, 3072), labels shape = (50000,)
Test data shape = (10000, 3072), labels shape = (10000,)
Batch Data = [[ 0.18431373  0.12941176  0.1372549  ... -0.30196078 -0.33333333
  -0.36470588]
 [-0.01176471 -0.01960784 -0.02745098 ... -0.22352941 -0.23137255
  -0.23921569]
 [ 0.40392157  0.41176471  0.44313725 ... -0.30196078 -0.31764706
  -0.3254902 ]
 ...
 [ 0.44313725  0.45882353  0.51372549 ...  0.49019608  0.49019608
   0.45882353]
 [-0.22352941 -0.18431373 -0.11372549 ...  0.14509804  0.11372549
   0.14509804]
 [-0.52156863 -0.45882353 -0.3254902  ... -0.30196078 -0.29411765
  -0.31764706]], Batch Labels = [0 1 8 4 7 8 3 9 6 5]


In [3]:
# Define x and y
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

# We need to reshape x into a image with RGB channels, aka. 32 * 32 * 3 matrix
x_image = tf.transpose(tf.reshape(x, [-1, 3, 32, 32]), perm = [0, 2, 3, 1])

# Define a Convoluational Layer and a Pooling Layer defined as below
conv1 = tf.layers.conv2d(x_image, 
                         32, # Number of output channels
                         (3, 3), # Kernel size
                         padding = "same", # Output will have the same size as input
                         activation = tf.nn.relu,
                         name = "conv1")
pooling1 = tf.layers.max_pooling2d(conv1,
                                   (2, 2), # Kernel size
                                   (2, 2), # Stride
                                   name = "pool1")
# Pooling1 size will be 16 * 16 * 32

conv2 = tf.layers.conv2d(pooling1, 
                         32, # Number of output channels
                         (3, 3), # Kernel size
                         padding = "same", # Output will have the same size as input
                         activation = tf.nn.relu,
                         name = "conv2")
pooling2 = tf.layers.max_pooling2d(conv2,
                                   (2, 2), # Kernel size
                                   (2, 2), # Stride
                                   name = "pool2")
# Pooling2 size will be 8 * 8 * 32

conv3 = tf.layers.conv2d(pooling2, 
                         32, # Number of output channels
                         (3, 3), # Kernel size
                         padding = "same", # Output will have the same size as input
                         activation = tf.nn.relu,
                         name = "conv3")
pooling3 = tf.layers.max_pooling2d(conv2,
                                   (2, 2), # Kernel size
                                   (2, 2), # Stride
                                   name = "pool3")
# Pooling2 size will be 4 * 4 * 32

flatten = tf.layers.flatten(pooling3) # size of flatten is [None, 512]
z = tf.layers.dense(flatten, 10)
y_predict = tf.argmax(z, 1)

# Calculate loss and accuracy
loss_crossEntropy = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = z)
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_predict, y), tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss_crossEntropy)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [6]:
print("3. Apply Basic Convolutional Network model to CIFAR-10 Data.")
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100
train_data.reset()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss_crossEntropy, accuracy, train_op],
                                        feed_dict = {x: batch_data, y: batch_labels})
        # Every 500 steps print loss and accuracy
        if (i + 1) % 100 == 0:
            print('[Train] step: %d, loss: %4.5f, accuracy: %4.5f' \
                 % (i + 1, loss_val, acc_val))
        
        # Every 5,000 steps run model against test model for accuracy
        if (i + 1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_accuracies = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_accuracies.append(test_acc_val)
            test_accuracy = np.mean(all_test_accuracies)
            print('[Test ] step: %d, accuracy: %4.5f' % (i + 1, test_accuracy))


3. Apply Basic Convolutional Network model to CIFAR-10 Data.
[Train] step: 100, loss: 1.49437, accuracy: 0.50000
[Train] step: 200, loss: 1.76098, accuracy: 0.35000
[Train] step: 300, loss: 1.38562, accuracy: 0.50000
[Train] step: 400, loss: 1.87712, accuracy: 0.35000
[Train] step: 500, loss: 1.32722, accuracy: 0.45000
[Train] step: 600, loss: 1.30085, accuracy: 0.55000
[Train] step: 700, loss: 1.24711, accuracy: 0.60000
[Train] step: 800, loss: 0.88515, accuracy: 0.55000
[Train] step: 900, loss: 1.31737, accuracy: 0.50000
[Train] step: 1000, loss: 1.30930, accuracy: 0.40000
[Test ] step: 1000, accuracy: 0.55400
[Train] step: 1100, loss: 1.50381, accuracy: 0.60000
[Train] step: 1200, loss: 1.10677, accuracy: 0.70000
[Train] step: 1300, loss: 1.12472, accuracy: 0.55000
[Train] step: 1400, loss: 1.00951, accuracy: 0.80000
[Train] step: 1500, loss: 1.13758, accuracy: 0.50000
[Train] step: 1600, loss: 0.87188, accuracy: 0.70000
[Train] step: 1700, loss: 1.15628, accuracy: 0.55000
[Train] s